In [ ]:
import os
import cv2

os.environ["PATH"] += os.pathsep + r"D:\Tessaract\Projects API\twitter bot\twitter_bot\video generation\ffmpeg\bin"

def augment_video(video_path, times):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    out = cv2.VideoWriter("first_output.mp4", fourcc, fps, (width, height))

    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    if len(frames) < 7:  
        print("Not enough frames to process smoothly.")
        return

    frames = frames[5:]

    sequence = []
    for i in range(times):
        if i % 2 == 0:
            sequence.extend(frames)  
        else:
            sequence.extend(frames[::-1])

    for frame in sequence:
        out.write(frame)

    out.release()
    print(f"Video saved as first_output.mp4 with {times} sequence(s).")


In [ ]:
from moviepy import VideoFileClip, AudioFileClip 
import math


def increase_template_video_length(audio_path, video_path):
    
    audio = AudioFileClip(audio_path)
    video = VideoFileClip(video_path)
    
    audio_length = audio.duration
    video_length = video.duration
    print(audio_length)
    
    # NOW, MAKING THE VIDEO EQUAL TO THE AUDIO LENGTH....
    
    if audio_length > video_length:
        max_video_length = math.ceil(audio_length)
        
        augment_video(video_path, max_video_length)
        
        second_video = VideoFileClip('first_output.mp4')
        second_video.audio = audio.subclipped(0, audio.duration)

        trimmed_video = second_video.subclipped(0, audio_length)


        trimmed_video.write_videofile("trimmed_output.mp4", codec="libx264", audio_codec="aac")
        


video_path = "avatar_forward.mp4"
audio_path = "sample.mp3"

        
increase_template_video_length(audio_path, video_path)
        

In [ ]:
import os
import cv2
import moviepy as mp
import whisper

os.environ["PATH"] += os.pathsep + r"D:\Tessaract\Projects API\twitter bot\twitter_bot\video_generation\ffmpeg\bin"

def overlay_subtitles(video_path, output_path, subtitle_chunks):
    video = mp.VideoFileClip(video_path)

 
    def add_overlay(frame):
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        current_time = video.reader.pos / video.fps  # Accurate frame time
        current_text = ""
        for chunk in subtitle_chunks:
            if chunk['start'] <= current_time <= chunk['end']:
                current_text = chunk['text']
                break

        if current_text:
            font_scale = 1
            font_thickness = 2
            text_color = (255, 255, 255)  # White text
            shadow_color = (0, 0, 0)  # Black shadow
            font = cv2.FONT_HERSHEY_SIMPLEX

            text_size, _ = cv2.getTextSize(current_text, font, font_scale, font_thickness)
            text_x = (video.w - text_size[0]) // 2
            text_y = video.h - 50  # Position at the bottom center

            # Shadow effect
            cv2.putText(frame_bgr, current_text, (text_x + 2, text_y + 2), font, font_scale, shadow_color, font_thickness + 1, cv2.LINE_AA)
            cv2.putText(frame_bgr, current_text, (text_x, text_y), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

        return cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

    modified_video = video.image_transform(add_overlay)
    modified_video.write_videofile(output_path, codec="libx264", fps=video.fps, audio_codec="aac")



def extract_subtitles(audio_path, shift_time=-0.1):
    model = whisper.load_model("small")
    result = model.transcribe(audio_path)
    subtitles = []

    for segment in result["segments"]:
        start_time = max(0, segment["start"] + shift_time)
        end_time = segment["end"] + shift_time
        text = segment["text"]

        mid_time = (start_time + end_time) / 2  
        words = text.split()

        if len(words) > 1:
            mid_index = len(words) // 2  
            first_half = " ".join(words[:mid_index])
            second_half = " ".join(words[mid_index:])

            subtitles.append({"start": start_time, "end": mid_time, "text": first_half})
            subtitles.append({"start": mid_time, "end": end_time, "text": second_half})
        else:
            subtitles.append({"start": start_time, "end": end_time, "text": text})

    return subtitles


video_path = "trimmed_output.mp4"  
audio_path = "sample.mp3"
output_video_path = "final_output.mp4"


subtitles = extract_subtitles(audio_path)
overlay_subtitles(video_path, output_video_path, subtitles)


In [ ]:
from moviepy import VideoFileClip, AudioFileClip, CompositeAudioClip

def add_background_music(video_path, music_path, output_path, music_volume=0.05):
    video = VideoFileClip(video_path)

    # Load the background music and set volume to 20%
    bg_music = AudioFileClip(music_path).with_volume_scaled(music_volume)

    # Set music duration to match the video length
    bg_music = bg_music.with_duration(video.duration)

    # Combine the video audio and background music
    final_audio = CompositeAudioClip([video.audio, bg_music])

    # Set the final audio to the video
    final_video = video.with_audio(final_audio)

    # Export the final video
    final_video.write_videofile(output_path, codec="libx264", audio_codec="aac")

# Example usage
video_path = "final_output.mp4"
music_path = "bg_voice_basketball.mp3"
output_path = "final_video_with_music.mp4"

add_background_music(video_path, music_path, output_path)
